## MNIST Training with MXNet and Gluon

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using MXNet and the Gluon API.



In [1]:
import os
import boto3
import sagemaker
from sagemaker.mxnet import MXNet
from mxnet import gluon
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

## Download training and test data

In [2]:
gluon.data.vision.MNIST('./data/train', train=True)
gluon.data.vision.MNIST('./data/test', train=False)

## Uploading the data

We use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value `inputs` identifies the location -- we will use this later when we start the training job.

In [3]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-mnist')

## Implement the training function

We need to provide a training script that can run on the SageMaker platform. The training scripts are essentially the same as one you would write for local training, except that you need to provide a `train` function. The `train` function will check for the validation accuracy at the end of every epoch and checkpoints the best model so far, along with the optimizer state, in the folder `/opt/ml/checkpoints` if the folder path exists, else it will skip the checkpointing. When SageMaker calls your function, it will pass in arguments that describe the training environment. Check the script below to see how this works.

The script here is an adaptation of the [Gluon MNIST example](https://github.com/apache/incubator-mxnet/blob/master/example/gluon/mnist.py) provided by the [Apache MXNet](https://mxnet.incubator.apache.org/) project. 

In [4]:
!cat 'mnist.py'

from __future__ import print_function

import argparse
import logging
import os

import mxnet as mx
from mxnet import gluon, autograd
from mxnet.gluon import nn
import numpy as np
import json
import time


logging.basicConfig(level=logging.DEBUG)

# ------------------------------------------------------------ #
# Training methods                                             #
# ------------------------------------------------------------ #


def train(args):
    # SageMaker passes num_cpus, num_gpus and other args we can use to tailor training to
    # the current container environment, but here we just use simple cpu context.
    ctx = mx.cpu()

    # retrieve the hyperparameters we set in notebook (with some defaults)
    batch_size = args.batch_size
    epochs = args.epochs
    learning_rate = args.learning_rate
    momentum = args.momentum
    log_interval = args.log_interval

    num_gpus = int(os.environ['SM_NUM_GPUS'])
    current_host = args.current_host
    hosts = args.hosts
 

## Run the training script on SageMaker

The ```MXNet``` class allows us to run our training function on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. In this case we will run our training job on a single c4.xlarge instance. 

In [8]:
m = MXNet("mnist.py",
          role=role,
          train_instance_count=2,
          train_instance_type="ml.p2.xlarge",
          framework_version="1.4.1",
          py_version="py3",
          distributions={'parameter_server': {'enabled': True}},
          hyperparameters={'batch-size': 100,
                           'epochs': 20,
                           'learning-rate': 0.1,
                           'momentum': 0.9, 
                           'log-interval': 100})

After we've constructed our `MXNet` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [ ]:
m.fit(inputs)

2019-12-12 04:36:58 Starting - Starting the training job...
2019-12-12 04:36:59 Starting - Launching requested ML instances......
2019-12-12 04:38:05 Starting - Preparing the instances for training.........
2019-12-12 04:39:47 Downloading - Downloading input data...
2019-12-12 04:40:19 Training - Downloading the training image..2019-12-12 04:40:40,809 sagemaker-containers INFO     Imported framework sagemaker_mxnet_container.training
2019-12-12 04:40:40,839 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1","algo-2"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{"batch-size":100,"epochs":20,"learning-rate":0.1,"log-interval":100,"momentum":0.9}', 'SM_USER_ENTRY_POINT': 'mnist.py', 'SM_FRAMEWORK_PARAMS': '{"sagemaker_parameter_server_enabled":true}', 'SM_RESOURCE_CONFIG': '{"current_host":"algo-2","hosts":["algo-1","algo-2"],"network_interface_name":"eth0"}', 'SM_INPUT_DATA_CONFIG': '{"training":{"RecordWrapperType":"None","S3Distributio


2019-12-12 04:40:40 Training - Training image download completed. Training in progress.Installing collected packages: mnist
  Running setup.py install for mnist: started
    Running setup.py install for mnist: finished with status 'done'
You should consider upgrading via the 'pip install --upgrade pip' command.
2019-12-12 04:40:46,419 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {
        "sagemaker_parameter_server_enabled": true
    },
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host": "algo-2",
    "framework_module": "sagemaker_mxnet_container.training:main",
    "hosts": [
        "algo-1",
        "algo-2"
    ],
    "hyperparameters": {
        "batch-size": 100,
        "log-interval": 100,
        "learning-rate": 0.1,
        "epochs": 20,
        "momentum": 0.9
    },
    "input_config_dir": "/opt/ml/input/config",
    "input_data_config": {
        "trainin

[Epoch 5] Training: accuracy=0.986100
[Epoch 5] Validation: accuracy=0.970400
[Epoch 6 Batch 100] Training: accuracy=0.989307, 16137.524528 samples/s
[Epoch 6 Batch 200] Training: accuracy=0.989154, 11684.925477 samples/s
[Epoch 6] Training: accuracy=0.989067
[Epoch 6] Training: accuracy=0.988300
[Epoch 6] Validation: accuracy=0.968600
[Epoch 6] Validation: accuracy=0.968600
[Epoch 7 Batch 100] Training: accuracy=0.988515, 9113.099402 samples/s
[Epoch 7 Batch 200] Training: accuracy=0.989900, 15359.250037 samples/s
[Epoch 7 Batch 100] Training: accuracy=0.989010, 12792.192265 samples/s
[Epoch 7 Batch 200] Training: accuracy=0.989154, 14271.194284 samples/s
[Epoch 7] Training: accuracy=0.989367
[Epoch 7] Training: accuracy=0.988633
[Epoch 7] Validation: accuracy=0.970700
[Epoch 7] Validation: accuracy=0.970700
[Epoch 8 Batch 100] Training: accuracy=0.990297, 20188.217174 samples/s
[Epoch 8 Batch 100] Training: accuracy=0.990099, 17906.775392 samples/s
[Epoch 8 Batch 200] Training: accur

After training, we use the MXNet object to build and deploy an MXNetPredictor object. This creates a SageMaker endpoint that we can use to perform inference. 

This allows us to perform inference on json encoded multi-dimensional arrays. 

In [ ]:
predictor = m.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a 'data' variable in this notebook, which we can then pass to the mxnet predictor. 

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

The predictor runs inference on our input data and returns the predicted digit (as a float value, so we convert to int for display).

In [ ]:
response = predictor.predict(data)
print(int(response))

## Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [ ]:
predictor.delete_endpoint()